Antes de empezar descarga las databases reducidas y subelas directamente en los archivos del colab

In [3]:
!pip install fastapi uvicorn pyngrok nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 5.8 MB/s eta 0:00:00


In [5]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import uvicorn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from fastapi import FastAPI, HTTPException
import nest_asyncio
from pyngrok import ngrok
from typing import List

# Cargar los datos en dataframes desde las rutas funcionales
movies_data = pd.read_csv('/content/movies_dataset_reducido.csv', low_memory=False)
credits_data = pd.read_csv('/content/credits_reducido.csv', low_memory=False)
# Muestra la forma de los datasets para verificar la carga
print("Movies Data Shape:", movies_data.shape)
print("Credits Data Shape:", credits_data.shape)

# Muestra las primeras filas de ambos datasets
print("\nMovies Data Head:")
print(movies_data.head())
print("\nCredits Data Head:")
print(credits_data.head())

# Información detallada de los datasets
print("\nMovies Data Info:")
movies_data.info()
print("\nCredits Data Info:")
credits_data.info()

# Filtrar y limpiar filas sin fecha de lanzamiento en movies_data
movies_data_cleaned = movies_data.dropna(subset=['release_date'])

# Transformación de valores nulos en revenue y budget a 0
movies_data_cleaned['return'] = np.where(
    (movies_data_cleaned['budget'] > 0),
    movies_data_cleaned['revenue'] / movies_data_cleaned['budget'],
    0)


# Convertir la fecha de lanzamiento al formato AAAA-mm-dd y extraer el año de lanzamiento
movies_data_cleaned['release_date'] = pd.to_datetime(movies_data_cleaned['release_date'], errors='coerce')
movies_data_cleaned['release_year'] = movies_data_cleaned['release_date'].dt.year

# Crear columna 'return' (retorno de inversión) usando revenue / budget y asignar 0 donde no es calculable
movies_data_cleaned['budget'] = pd.to_numeric(movies_data_cleaned['budget'], errors='coerce')
movies_data_cleaned['revenue'] = pd.to_numeric(movies_data_cleaned['revenue'], errors='coerce')
movies_data_cleaned['return'] = np.where(movies_data_cleaned['budget'] > 0, movies_data_cleaned['revenue'] / movies_data_cleaned['budget'], 0)

# Eliminar columnas innecesarias en movies_data
columns_to_drop = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
movies_data_cleaned = movies_data_cleaned.drop(columns=columns_to_drop)

# Desanidar columnas anidadas en movies_data
def desanidar_columna(columna):
    try:
        return ast.literal_eval(columna) if isinstance(columna, str) else []
    except ValueError:
        return []

movies_data_cleaned['belongs_to_collection'] = movies_data_cleaned['belongs_to_collection'].apply(desanidar_columna)
movies_data_cleaned['production_companies'] = movies_data_cleaned['production_companies'].apply(desanidar_columna)
movies_data_cleaned['production_countries'] = movies_data_cleaned['production_countries'].apply(desanidar_columna)

# Comprobación del dataset transformado
print("\nMovies Data Cleaned Info:")
print(movies_data_cleaned.info())
print("\nMovies Data Cleaned Head:")
print(movies_data_cleaned.head())


Movies Data Shape: (450, 24)
Credits Data Shape: (450, 3)

Movies Data Head:
   adult                              belongs_to_collection    budget  \
0  False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1  False                                                NaN  65000000   
2  False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3  False                                                NaN  16000000   
4  False  {'id': 96871, 'name': 'Father of the Bride Col...         0   

                                              genres  \
0  [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1  [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2  [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                     [{'id': 35, 'name': 'Comedy'}]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709           

In [28]:
!ls

credits_reducido.csv  drive  movies_dataset_reducido.csv  sample_data


In [6]:
!ngrok config add-authtoken 2omes6KPch2ZGwnisTmUHU9kZZP_4BtnMwEA5Ypx1noHBuGDM

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [29]:
movies_data = movies_data.merge(credits_data, on='id')

In [30]:
movies_data['overview'] = movies_data['overview'].fillna('')

In [31]:
def obtener_informacion_principal(x, key):
    try:
        items = ast.literal_eval(x)
        return ' '.join([d[key] for d in items if key in d])
    except:
        return ''

In [32]:
movies_data['actors'] = movies_data['cast'].apply(lambda x: obtener_informacion_principal(x, 'name'))
movies_data['director'] = movies_data['crew'].apply(lambda x: obtener_informacion_principal(x, 'name') if 'Director' in x else '')
movies_data['genres'] = movies_data['genres'].apply(lambda x: obtener_informacion_principal(x, 'name'))
movies_data['production_companies'] = movies_data['production_companies'].apply(lambda x: obtener_informacion_principal(x, 'name'))

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-4' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

In [33]:
movies_data['content'] = movies_data['overview'] + ' ' + movies_data['genres'] + ' ' + \
                         movies_data['actors'] + ' ' + movies_data['director'] + ' ' + \
                         movies_data['production_companies']

In [34]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_data['content'])

In [35]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [36]:
indices = pd.Series(movies_data.index, index=movies_data['title']).drop_duplicates()

In [37]:
def recomendar_peliculas(title, cosine_sim=cosine_sim, df=movies_data, indices=indices):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

In [38]:
pelicula = "Toy Story"
print(f"Películas recomendadas para '{pelicula}':\n")
print(recomendar_peliculas(pelicula))

Películas recomendadas para 'Toy Story':

351                Forrest Gump
314    The Shawshank Redemption
51             Mighty Aphrodite
31               Twelve Monkeys
309           The Swan Princess
147                   Apollo 13
73                 Bed of Roses
205                  Waterworld
435                        Dave
430                   Coneheads
Name: title, dtype: object


In [39]:
app = FastAPI()

In [10]:
!pip install pyngrok

import nest_asyncio
from pyngrok import ngrok

nest_asyncio.apply()

# Change 'port' to 'addr' in ngrok.connect
public_url = ngrok.connect(addr="8000")
print("URL pública de la API:", public_url)


URL pública de la API: NgrokTunnel: "https://53c1-35-199-175-165.ngrok-free.app" -> "http://localhost:8000"


In [17]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def root():
    return {"message": "Hello, World!"}

In [41]:
# Cierra todos los túneles activos
ngrok.kill()

# Reinicia un nuevo túnel en el puerto 8000
public_url = ngrok.connect(addr="8000")
print("URL pública de la API:", public_url)
#Este codigo va a reutilizarse cada vez que se intente deployar el servidor en la ultima celda

URL pública de la API: NgrokTunnel: "https://c82a-35-199-175-165.ngrok-free.app" -> "http://localhost:8000"


In [43]:
from fastapi import FastAPI

app = FastAPI()

# Endpoint de ejemplo para obtener un ítem
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}


In [44]:
@app.get("/recomendar/{title}")
def get_recommendations(title: str):
    try:
        recomendaciones = recomendar_peliculas(title)
        return {"title": title, "recommendations": recomendaciones}
    except ValueError as e:
        raise HTTPException(status_code=404, detail=str(e))


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-13' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", 

In [45]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Permitir que FastAPI corra asincrónicamente en el notebook
nest_asyncio.apply()

# Change 'port' to 'addr' in ngrok.connect
public_url = ngrok.connect(addr=8000)  # Use 'addr' instead of 'port'
print("https://c82a-35-199-175-165.ngrok-free.app", public_url)


# Ejecutar el servidor de FastAPI
uvicorn.run(app, host="0.0.0.0", port=8000)

https://c82a-35-199-175-165.ngrok-free.app NgrokTunnel: "https://a87d-35-199-175-165.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [188]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [188]


Esta celda se tiene que mantener en ejecución para que funcione el servidor, asi que hay que ejecutarla al ultimo.

esta celda dara error si se trata de ejecutar dos veces con el mismo tunel ya que expira una vez usado. para seguir provandola uno necesita ejecutar la celda anterior con el siguente codigo y reempalzar el url dentro de print por el que te da el codigo a continuacion:  
"# Cierra todos los túneles activos

ngrok.kill()

# Reinicia un nuevo túnel en el puerto 8000

public_url = ngrok.connect(addr="8000")

print("URL pública de la API:", public_url)"